




# <h1 style='background:#0B2265; border:#A71930; border-width: thick; border-style: solid; color:white'><center>Water Quality : EDA</center></h1> 

# Loading

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt        
import seaborn as sns
import missingno

import warnings
warnings.filterwarnings("ignore")

# <h1 style='background:#0B2265; border:#A71930; border-width: thick; border-style: solid; color:white'><center> 1st Down : Meaningful to Having Null? Nope!   </center></h1> 

# Null Check

In [ ]:
# Parette Color Set

NY_GIANTS_Color = ["#0B2265", "#A71930", "#A5ACAF"]
NY_GIANTS = {0 : NY_GIANTS_Color[0], 1 : NY_GIANTS_Color[1], 
             "Not Drinkable" : NY_GIANTS_Color[0], "Drinkable" : NY_GIANTS_Color[1]}

# Loading Data
df = pd.read_csv('/kaggle/input/water-potability/water_potability.csv')

# Missing Value Check Visulization
missingno.matrix(df, color =(0.04, 0.13, 0.40))

I has doubt that observation, which has null, might be not drinkable.

If I am a warter inspector, I would not give OK sign to data on full inspection.

In [ ]:
# For understand easier
df.loc[df["Potability"] == 1, "Drink"] = "Drinkable"
df.loc[df["Potability"] == 0, "Drink"] = "Not Drinkable"

In [ ]:
sns.countplot(x='Drink', data=df, palette=NY_GIANTS)

In [ ]:
no_drink = df['Potability'][df.Potability == 0].count()
ok_drink = df['Potability'][df.Potability == 1].count()

no_drink_percent = no_drink/(ok_drink+no_drink)

print ("Percentage of Not Drinkable Percentage: {:.4%}%".format(no_drink_percent))

In [ ]:
# Null Check
df_null = df
df_null['Has_null'] = df_null.isnull().any(axis=1)
df_null

sns.countplot(x="Has_null", hue="Drink", data=df_null, palette=NY_GIANTS)

Ok... ratio looks like... I would be wrong.

In [ ]:
no_null_ok_drink = df_null['Potability'][(df_null.Has_null == False) & (df.Potability == 1)].count()
no_null_no_drink = df_null['Potability'][(df_null.Has_null == False) & (df.Potability == 0)].count()

has_null_ok_drink = df_null['Potability'][(df_null.Has_null == True) & (df.Potability == 1)].count()
has_null_no_drink = df_null['Potability'][(df_null.Has_null == True) & (df.Potability == 0)].count()

no_null_percent = no_null_no_drink/(no_null_no_drink+no_null_ok_drink)
has_null_percent = has_null_no_drink/(has_null_no_drink+has_null_ok_drink)

print ("No Null index - Percentage of Potability: {:.4%}%".format(no_null_percent))
print ("Has Null index - Percentage of Potability: {:.4%}%".format(has_null_percent))

Yes I am wrong on my first hypothesis. 

I think I can put mean value on all null. Because Null would not impact on overall view.

In [ ]:
df = df.drop('Has_null', axis=1)
df = df.fillna(df.mean())
missingno.matrix(df, color =(0.04, 0.13, 0.40))

No null, proceed next step.

# <h1 style='background:#0B2265; border:#A71930; border-width: thick; border-style: solid; color:white'><center> 2nd Down : Bad News -- Too Low Correlation</center></h1> 

# Feature Correlation

In [ ]:
sns.pairplot(df, hue='Drink', palette=NY_GIANTS)

These graphs are like being hit by a shotgun.

I could not see any clue for now.

In [ ]:
sns.set_theme(style="white")

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(10, 240, n=8)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,vmax=0.2, vmin=-0.2,
            annot=True, square=True, linewidths=0.5, cbar_kws={"shrink": .5})

Potability(=Drinkable) shows very close on zero. Show no relationship with other features.

This is bad news.

# <h1 style='background:#0B2265; border:#A71930; border-width: thick; border-style: solid; color:white'><center> 3rd Down : Why is "Water Quality" dataset hard to solve?</center></h1> 

# Features Distribution

In [ ]:
fig = plt.figure(figsize=(15,20))

fig=plt.subplot(3,3,1)
sns.histplot(df, x=df.columns[0], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,2)
sns.histplot(df, x=df.columns[1], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,3)
sns.histplot(df, x=df.columns[2], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,4)
sns.histplot(df, x=df.columns[3], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,5)
sns.histplot(df, x=df.columns[4], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,6)
sns.histplot(df, x=df.columns[5], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,7)
sns.histplot(df, x=df.columns[6], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,8)
sns.histplot(df, x=df.columns[7], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,9)
sns.histplot(df, x=df.columns[8], hue="Drink", element="step", palette=NY_GIANTS)

Why is "Water Quality" dataset hard to solve?

Because 
1. distribution between "Drinkable" and "Not Drinkable" are overlaped.
2. They shows almost same average and standard distribution.
3. Features are two ways : Drinkable warter should has not higher and also not lower on all factors.

On reason 3, for example, "PH" too low value on PH could not drinkable, and also too high value could not drinkable)

But shape of distribution might be chance to shold prolblem. 

In [ ]:
df_ss = df
df_ss = df_ss.drop('Potability', axis=1)
df_ss = df_ss.drop('Drink', axis=1)
df_ss = (df_ss - df_ss.mean()) / df_ss.std()
df_ss = df_ss.abs()
df_ss['Drink'] = df['Drink']

sns.pairplot(df_ss, hue='Drink', palette=NY_GIANTS)

In [ ]:
sns.set_theme(style="white")

# Compute the correlation matrix
corr = df_ss.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(10, 240, n=8)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,vmax=0.2, vmin=-0.2,
            annot=True, square=True, linewidths=0.5, cbar_kws={"shrink": .5})

In [ ]:
fig = plt.figure(figsize=(15,20))

fig=plt.subplot(3,3,1)
sns.histplot(df_ss, x=df_ss.columns[0], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,2)
sns.histplot(df_ss, x=df_ss.columns[1], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,3)
sns.histplot(df_ss, x=df_ss.columns[2], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,4)
sns.histplot(df_ss, x=df_ss.columns[3], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,5)
sns.histplot(df_ss, x=df_ss.columns[4], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,6)
sns.histplot(df_ss, x=df_ss.columns[5], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,7)
sns.histplot(df_ss, x=df_ss.columns[6], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,8)
sns.histplot(df_ss, x=df_ss.columns[7], hue="Drink", element="step", palette=NY_GIANTS)

fig=plt.subplot(3,3,9)
sns.histplot(df_ss, x=df_ss.columns[8], hue="Drink", element="step", palette=NY_GIANTS)

# <h1 style='background:#0B2265; border:#A71930; border-width: thick; border-style: solid; color:white'><center> 4th Down : Prediction</center></h1> 

# Model

In [ ]:
df_ss = df_ss.drop('Drink', axis=1)
df_ss['Potability'] = df['Potability']

In [ ]:
data = df_ss.iloc[:,0:-1].to_numpy()
target = df_ss.iloc[:,-1:]

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.1, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate

LGBM

In [ ]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("Train Accuracy : ", np.mean(scores['train_score']))
print("Test Accuracy : ",np.mean(scores['test_score']))

XG Boost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print("Train Score : ", np.mean(scores['train_score']))
print("Test Score : ", np.mean(scores['test_score']))

Hist Gradient Boosting

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

print("Train Score : ",np.mean(scores['train_score']))
print("Test Score : ",np.mean(scores['test_score']))

# <h1 style='background:#0B2265; border:#A71930; border-width: thick; border-style: solid; color:white'><center> Loss Game! Conclusion </center></h1> 

# Conclusion

I hope you - do not too deep dive this dataset for modeling, only what I have was overfitted result.

Because distribution on all features are overlapped, could not generate meaningful model.

If you optimize hyper parameters, you may +10% score than me, but I think that's it.